In [6]:
from pynq import Overlay
import numpy as np
import time
from pynq import MMIO

ol = Overlay("/home/xilinx/jupyter_notebooks/matrix_mult.bit")

matmul = ol.matrix_mult_0
bram_a = ol.axi_bram_ctrl_2
bram_b = ol.axi_bram_ctrl_1
bram_prod = ol.axi_bram_ctrl_0

# Define the size of the matrices
dimension = 16 # in this case it's 16x16 so we only need 1 variable instead of two


# Generate random matrices with int32 values
mat_a = np.random.randint(0, 5, size=(dimension,dimension), dtype=np.int32)
print("Matrix A:\n", mat_a)
mat_b = np.random.randint(0, 5, size=(dimension,dimension), dtype=np.int32)
print("\nMatrix B:\n", mat_b)

start_time_n = time.time()
npprod = np.dot(mat_a,mat_b)
elapsed_time_n = time.time() - start_time_n
print(f"\nExpected matrix:\n {npprod}")
print(f"\nSoftware Elapsed time: {elapsed_time_n} seconds\n")




offset = 0
MatA = getattr(mat_a, "tolist", lambda: mat_a)()
for i in range(dimension ):
  for x in range(dimension):
    bram_a.write(offset, MatA[i][x])
    offset = offset + 4


offset = 0
matrixA_Read = np.zeros((dimension, dimension), dtype=np.int32)
for x in range(dimension):
  for y in range(dimension):
    matrixA_Read[x][y]=bram_a.read(offset)
    offset = offset + 4
print("\nA Matrix:\n",matrixA_Read)

AComp = mat_a == matrixA_Read
print("\nComparison:\n",AComp)






offsett = 0
MatB = getattr(mat_b, "tolist", lambda: mat_b)()
for x in range(dimension):
  for y in range(dimension):
    bram_b.write(offsett, MatB[x][y])
    offsett = offsett + 4

#matmul operation
start_time = time.time()


offsett = 0
matrixB_Read = np.zeros((dimension, dimension), dtype=np.int32)
for x in range(dimension):
  for y in range(dimension):

    matrixB_Read[x][y]=bram_b.read(offsett)
    offsett = offsett + 4


print("\nB Matrix:\n",matrixB_Read)

BComp = mat_b == matrixB_Read
print("\nComparison:\n",BComp)


offset_p=0
prod = np.zeros((dimension, dimension), dtype=np.int32)
startTime = time.time()
matmul.register_map.CTRL.AP_START = 1

while (matmul.register_map.CTRL.AP_IDLE == 0):
    pass

for x in range(dimension):
    #print("i1")
    for y in range(dimension):
        #print("i2")
        
        prod[x][y] = bram_prod.read(offset_p)
        #print("Value: " + str(bram_prod.read(offset_p)))
        offset_p+=4
        
timeElapsed = time.time() - startTime

print(f"\nResultant matrix:\n {prod}")
print(f"\nHardware Elapsed time: {timeElapsed} seconds\n")

elementwise_comparison = prod == npprod
print(elementwise_comparison)


Matrix A:
 [[1 0 1 2 2 2 4 1 4 0 4 0 2 0 2 4]
 [3 0 1 3 2 4 2 1 4 2 0 1 2 3 2 0]
 [1 4 0 2 0 1 2 2 3 4 0 1 2 3 0 4]
 [1 1 0 1 3 2 1 2 4 3 3 0 4 4 0 0]
 [1 1 0 0 1 1 3 1 0 4 3 4 1 1 0 2]
 [0 4 3 2 4 0 4 0 3 2 1 0 4 4 0 4]
 [1 3 0 0 2 4 2 3 3 2 4 4 2 3 4 2]
 [3 2 2 1 3 1 3 2 0 1 3 3 0 0 4 2]
 [2 3 3 4 2 2 1 3 3 0 3 2 0 4 2 4]
 [1 1 3 3 0 4 2 0 0 3 3 4 1 4 3 0]
 [4 0 3 1 0 4 3 1 3 0 0 0 1 3 0 3]
 [2 3 0 1 2 4 0 1 1 2 3 3 3 1 0 3]
 [2 0 3 1 1 2 4 3 2 4 3 3 2 1 4 4]
 [4 1 1 0 4 1 1 4 3 1 1 3 3 3 2 1]
 [0 2 2 1 1 4 0 2 4 2 2 2 4 2 2 1]
 [3 2 3 4 0 2 1 1 2 0 0 3 2 1 3 4]]

Matrix B:
 [[0 1 0 4 2 3 4 2 4 4 3 0 1 0 3 1]
 [0 2 3 1 2 2 4 4 1 0 4 4 0 1 2 1]
 [1 1 2 2 1 4 1 1 0 4 2 2 0 0 4 3]
 [2 3 3 0 2 3 3 3 1 0 1 4 0 3 0 1]
 [3 4 3 1 1 1 2 2 2 0 1 1 4 0 3 3]
 [4 4 4 1 4 1 2 3 2 4 1 2 0 0 4 4]
 [0 2 0 0 3 1 4 0 0 3 0 4 3 3 2 1]
 [3 2 1 1 0 0 4 1 3 0 4 4 1 4 0 3]
 [4 2 0 0 1 1 1 4 1 4 1 1 3 1 4 0]
 [4 2 3 4 2 3 0 4 0 0 4 0 3 2 0 2]
 [1 0 1 0 1 4 0 2 4 4 0 1 0 1 2 3]
 [4 0 4 3 0 4 3 4 0 2 1 0 4 2 3